# Required Packages

In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import FastText
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import spacy
from IPython.display import Image
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import string
from time import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Data Acqusition

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path7 = '/content/drive/My Drive/capstone_drive/view_tables/view_posts_2024-04-07.csv'
file_path14 = '/content/drive/My Drive/capstone_drive/view_tables/view_posts_2024-04-14.csv'
file_path21 = '/content/drive/My Drive/capstone_drive/view_tables/view_posts_2024-04-21.csv'
file_path28 = '/content/drive/My Drive/capstone_drive/view_tables/view_posts_2024-04-28.csv'

# Read the CSV file into a DataFrame
df7 = pd.read_csv(file_path7)
df14 = pd.read_csv(file_path14)
df21 = pd.read_csv(file_path21)
df28 = pd.read_csv(file_path28)

In [2]:
file_path7 = '/content/drive/My Drive/capstone/view_posts_2024-04-07.csv'
df7 = pd.read_csv(file_path7)

NameError: name 'pd' is not defined

In [4]:
file_path7 = '/content/drive/My Drive/capstone/Week_04-07-2024_similarity'
df7 = pd.read_csv(file_path7)

In [9]:

file_path21 = '/content/drive/My Drive/capstone/view_posts_2024-04-21.csv'
file_path28 = '/content/drive/My Drive/capstone/view_posts_2024-04-28.csv'

# Read the CSV file into a DataFrame
df21 = pd.read_csv(file_path21)
df28 = pd.read_csv(file_path28)

# Overview of Data

In [ ]:
df7.head()

,post_id,post_title,post_body,post_date,tag
0,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,c#
1,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,mqttnet
2,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,maui-blazor
3,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,mqtt
4,78277808,How to use multiple authentication in ASP.NET ...,<p>I have an application that uses azure AD B2...,2024-04-05 04:49:07+00:00,azure


In [ ]:
df14.head()

,post_id,post_title,post_body,post_date,tag
0,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,python
1,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,python
2,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,python
3,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,wsgi
4,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,wsgi


# Text Processing Functions

In [28]:
def extract_code(text):
    # Regular expression pattern to match code blocks enclosed in <code> tags
    pattern = r"<code>(.*?)</code>"
    # Find all code snippets in the text
    code_snippets = re.findall(pattern, text, flags=re.DOTALL)
    # Join the code snippets into a single string
    code = '\n'.join(code_snippets)
    return code

# Function to clean text
def clean_text(text):
    if isinstance(text, str):  # Check if text is a string
        # Remove HTML tags and code snippets
        cleaned_text = re.sub(r'<[^>]+>', '', text)
        cleaned_text = re.sub(r"<code>(.*?)</code>", '', cleaned_text, flags=re.DOTALL)
        # Remove special characters
        cleaned_text = re.sub(r'[^a-zA-Z\s.]+', '', cleaned_text)
        # Remove extra whitespaces
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        return cleaned_text.strip()
    else:
        return None # Return empty string for non-string values


#df['Cleaned_Comment'] = df['comment_text'].apply(clean_text)

In [ ]:
# Apply cleaning and extraction functions to 'Post Body' and 'Comment' columns
df7['Cleaned_Post_Body'] = df7['post_body'].apply(clean_text)
df7['Code'] = df7['post_body'].apply(extract_code)

df14['Cleaned_Post_Body'] = df14['post_body'].apply(clean_text)
df14['Code'] = df14['post_body'].apply(extract_code)

df21['Cleaned_Post_Body'] = df21['post_body'].apply(clean_text)
df21['Code'] = df21['post_body'].apply(extract_code)

df28['Cleaned_Post_Body'] = df28['post_body'].apply(clean_text)
df28['Code'] = df28['post_body'].apply(extract_code)

In [29]:
df7['Cleaned_Post_Body'] = df7['post_body'].apply(clean_text)
df7['Code'] = df7['post_body'].apply(extract_code)

In [11]:

df21['Cleaned_Post_Body'] = df21['post_body'].apply(clean_text)
df21['Code'] = df21['post_body'].apply(extract_code)

df28['Cleaned_Post_Body'] = df28['post_body'].apply(clean_text)
df28['Code'] = df28['post_body'].apply(extract_code)

In [ ]:
# Fill NaN with empty string if 'processed_text' has missing values
df7['Cleaned_Post_Body'] = df7['Cleaned_Post_Body'].fillna("")
df14['Cleaned_Post_Body'] = df14['Cleaned_Post_Body'].fillna("")
df21['Cleaned_Post_Body'] = df21['Cleaned_Post_Body'].fillna("")
df28['Cleaned_Post_Body'] = df28['Cleaned_Post_Body'].fillna("")

In [30]:
df7['Cleaned_Post_Body'] = df7['Cleaned_Post_Body'].fillna("")

In [12]:
df21['Cleaned_Post_Body'] = df21['Cleaned_Post_Body'].fillna("")
df28['Cleaned_Post_Body'] = df28['Cleaned_Post_Body'].fillna("")

In [ ]:
df7.head()

,post_id,post_title,post_body,post_date,tag,Cleaned_Post_Body,Code
0,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,c#,I have created a Server Android Blazor MAUI pr...,public async Task StartServer()\n{\n IPHost...
1,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,mqttnet,I have created a Server Android Blazor MAUI pr...,public async Task StartServer()\n{\n IPHost...
2,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,maui-blazor,I have created a Server Android Blazor MAUI pr...,public async Task StartServer()\n{\n IPHost...
3,78266342,Cannot Connect to MqttNet Server running on An...,<p>I have created a Server Android Blazor MAUI...,2024-04-03 08:33:01+00:00,mqtt,I have created a Server Android Blazor MAUI pr...,public async Task StartServer()\n{\n IPHost...
4,78277808,How to use multiple authentication in ASP.NET ...,<p>I have an application that uses azure AD B2...,2024-04-05 04:49:07+00:00,azure,I have an application that uses azure AD BC. I...,ConfigureServices\nservices.AddAuthentication(...


In [ ]:
# Display the DataFrame with cleaned text and extracted code snippets
df14.head()

,post_id,post_title,post_body,post_date,tag,Cleaned_Post_Body,Code
0,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,python,On an Ubuntu . environment Im trying to access...,ModuleNotFoundError: No module named 'cv2' \n&...
1,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,python,On an Ubuntu . environment Im trying to access...,ModuleNotFoundError: No module named 'cv2' \n&...
2,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,python,On an Ubuntu . environment Im trying to access...,ModuleNotFoundError: No module named 'cv2' \n&...
3,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,wsgi,On an Ubuntu . environment Im trying to access...,ModuleNotFoundError: No module named 'cv2' \n&...
4,78317620,Python WSGI/websocket server in Apache: Module...,"<p>On an Ubuntu 22.04 environment, I'm trying ...",2024-04-12 16:28:02+00:00,wsgi,On an Ubuntu . environment Im trying to access...,ModuleNotFoundError: No module named 'cv2' \n&...


In [13]:
df21.head()

,post_id,post_title,post_body,post_date,tag,Cleaned_Post_Body,Code
0,73163386,OOP is discouraged in Rust but there don't see...,<p>I am trying to write a 3d application in ru...,2022-07-29 07:56:13+00:00,rust,I am trying to write a d application in rust a...,"enum SceneObj {\n Plane(Plane),\n Sphere..."
1,73163386,OOP is discouraged in Rust but there don't see...,<p>I am trying to write a 3d application in ru...,2022-07-29 07:56:13+00:00,rust,I am trying to write a d application in rust a...,"enum SceneObj {\n Plane(Plane),\n Sphere..."
2,73163386,OOP is discouraged in Rust but there don't see...,<p>I am trying to write a 3d application in ru...,2022-07-29 07:56:13+00:00,oop,I am trying to write a d application in rust a...,"enum SceneObj {\n Plane(Plane),\n Sphere..."
3,73163386,OOP is discouraged in Rust but there don't see...,<p>I am trying to write a 3d application in ru...,2022-07-29 07:56:13+00:00,oop,I am trying to write a d application in rust a...,"enum SceneObj {\n Plane(Plane),\n Sphere..."
4,73163386,OOP is discouraged in Rust but there don't see...,<p>I am trying to write a 3d application in ru...,2022-07-29 07:56:13+00:00,3d,I am trying to write a d application in rust a...,"enum SceneObj {\n Plane(Plane),\n Sphere..."


In [14]:
df28.head()

,post_id,post_title,post_body,post_date,tag,Cleaned_Post_Body,Code
0,78364238,How to check api connection in post method in ...,<p>I'm developing an xamarin forms app and I u...,2024-04-22 06:06:07+00:00,xamarin.forms,Im developing an xamarin forms app and I use o...,public async Task&lt;Model.Result&gt; PostSto...
1,78364238,How to check api connection in post method in ...,<p>I'm developing an xamarin forms app and I u...,2024-04-22 06:06:07+00:00,mobile,Im developing an xamarin forms app and I use o...,public async Task&lt;Model.Result&gt; PostSto...
2,78364238,How to check api connection in post method in ...,<p>I'm developing an xamarin forms app and I u...,2024-04-22 06:06:07+00:00,android,Im developing an xamarin forms app and I use o...,public async Task&lt;Model.Result&gt; PostSto...
3,78364238,How to check api connection in post method in ...,<p>I'm developing an xamarin forms app and I u...,2024-04-22 06:06:07+00:00,c#,Im developing an xamarin forms app and I use o...,public async Task&lt;Model.Result&gt; PostSto...
4,78360901,Strapi send email after payment compelete,<p>Strapi send email after payment compelete</...,2024-04-21 08:35:55+00:00,reactjs,Strapi send email after payment compelete I ha...,yarn add @strapi/provider-email-sendgrid\n\nmo...


# Text Preprocessing Utilities

In [15]:
# Initialize SpaCy English model
nlp = spacy.load('en_core_web_sm')

# Function to extract code snippets from text
def extract_code(text):
    # Regular expression pattern to match code blocks enclosed in <code> tags
    pattern = r"<code>(.*?)</code>"
    # Find all code snippets in the text
    code_snippets = re.findall(pattern, text, flags=re.DOTALL)
    # Join the code snippets into a single string
    code = '\n'.join(code_snippets)
    return code

# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags and code snippets
        cleaned_text = re.sub(r'<[^>]+>', '', text)
        cleaned_text = re.sub(r"<code>(.*?)</code>", '', cleaned_text, flags=re.DOTALL)
        # Remove special characters
        cleaned_text = re.sub(r'[^a-zA-Z\s.]+', '', cleaned_text)
        # Remove extra whitespaces
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        return cleaned_text.strip()
    return None


# Download stopwords if not already done
nltk.download('stopwords')

# Create a custom list of programming keywords to add to stop words
programming_keywords = [
    "def", "function", "return", "var", "const", "let", "console",
    "print", "import", "class", "this", "new", "try", "catch", "init","py"
]

# Function to remove stop words, including custom-defined words
def remove_custom_stop_words(tokens, custom_stop_words):
    """
    Removes English stop words and additional custom stop words from a list of tokenized words.
    """
    # Get the list of English stop words
    base_stop_words = set(stopwords.words('english'))

    # Combine base stop words with custom-defined stop words
    combined_stop_words = base_stop_words.union(custom_stop_words)

    # Filter out stop words from the list of tokens
    filtered_tokens = [token for token in tokens if token.lower() not in combined_stop_words]

    return filtered_tokens

# Function to tokenize at the sentence level
def tokenize_sentences(text):
    if isinstance(text, str):
        # Split into sentences
        sentences = sent_tokenize(text)

        # Remove punctuation from each sentence
        sentences_no_punctuation = []
        for sentence in sentences:
            # Remove punctuation using string.punctuation
            cleaned_sentence = "".join(
                char for char in sentence if char not in string.punctuation
            )
            sentences_no_punctuation.append(cleaned_sentence.strip())

        return sentences_no_punctuation

    return []

# Function to lemmatize using SpaCy
def lemmatize(text):
    if isinstance(text, str):
        doc = nlp(text)
        return [token.lemma_ for token in doc]
    return []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
def group_tags_with_other_columns(df):
    """
    Groups DataFrame by 'post_id', aggregates 'tag' into a list,
    and retains other columns without aggregation.
    """
    # Group by 'post_id' and aggregate 'tag' into a list
    tags_aggregated = df.groupby('post_id')['tag'].apply(list).reset_index()

    # Get unique rows based on 'post_id' to retain other columns
    unique_rows = df.drop_duplicates(subset=['post_id']).drop('tag', axis=1)

    # Merge the aggregated 'tags' with the unique rows
    final_df = pd.merge(tags_aggregated, unique_rows, on='post_id')

    return final_df

In [31]:
df7 = group_tags_with_other_columns(df7)
# Apply the functions to the DataFrame
df7['Cleaned Post Body'] = df7['post_body'].apply(clean_text)
df7['Code'] = df7['post_body'].apply(extract_code)
df7['Simple Tokens'] = df7['Cleaned Post Body'].apply(lambda text: text.split())
df7['NLTK Tokens'] = df7['Cleaned Post Body'].apply(word_tokenize)
# df['SpaCy Tokens'] = df['Cleaned Post Body'].apply(tokenize_spacy)
df7['Sentences'] = df7['Cleaned Post Body'].apply(tokenize_sentences)
# df7['Lemmas'] = df7['Cleaned Post Body'].apply(lemmatize)
# df['Stems'] = df['Cleaned Post Body'].apply(stem)

# Display the DataFrame to check results
print(df7)

       post_id                                                tag  \
0        44834                       [namespaces, python, syntax]   
1       318563                [information-architecture, sitemap]   
2       454734      [timestamp, git, commit, git-rewrite-history]   
3       495553                                     [firefox, dns]   
4       606191                       [string, python-3.x, python]   
...        ...                                                ...   
8080  78286207  [azure-application-insights, azure-application...   
8081  78286210                                [spring-boot, java]   
8082  78286215            [mouseleave, javascript, panel, jquery]   
8083  78286216  [cmake, cmake, cmake, cmake, cmake, opencv, op...   
8084  78286227  [google-sheets, google-sheets, google-sheets, ...   

                                             post_title  \
0                     What does __all__ mean in Python?   
1      Generating a Visual Site Map of an Existing Si

In [ ]:
df14 = group_tags_with_other_columns(df14)
# Apply the functions to the DataFrame
df14['Cleaned Post Body'] = df14['post_body'].apply(clean_text)
df14['Code'] = df14['post_body'].apply(extract_code)
df14['Simple Tokens'] = df14['Cleaned Post Body'].apply(lambda text: text.split())
df14['NLTK Tokens'] = df14['Cleaned Post Body'].apply(word_tokenize)
# df['SpaCy Tokens'] = df['Cleaned Post Body'].apply(tokenize_spacy)
df14['Sentences'] = df14['Cleaned Post Body'].apply(tokenize_sentences)
df14['Lemmas'] = df14['Cleaned Post Body'].apply(lemmatize)
# df['Stems'] = df['Cleaned Post Body'].apply(stem)

# Display the DataFrame to check results
print(df14)

       post_id                                                tag  \
0       597687                      [refactoring, vim, variables]   
1       598951                                 [javascript, html]   
2       761194                   [abstract-class, oop, interface]   
3       763516                         [sqlite, schema, database]   
4       783818                            [exception, javascript]   
...        ...                                                ...   
8406  78322305  [spring-data-jpa, postgresql, java, spring-boo...   
8407  78322310                   [authlogic, ruby-on-rails, ruby]   
8408  78322315  [amazon-web-services, amazon-web-services, ama...   
8409  78322318                                    [eclipse, java]   
8410  78322320                  [flask, cors, python, chat, bots]   

                                             post_title  \
0          How to quickly change variable names in Vim?   
1     What is the easiest way to detect if at least .

In [17]:
df21 = group_tags_with_other_columns(df21)
# Apply the functions to the DataFrame
df21['Cleaned Post Body'] = df21['post_body'].apply(clean_text)
df21['Code'] = df21['post_body'].apply(extract_code)
df21['Simple Tokens'] = df21['Cleaned Post Body'].apply(lambda text: text.split())
df21['NLTK Tokens'] = df21['Cleaned Post Body'].apply(word_tokenize)
# df['SpaCy Tokens'] = df['Cleaned Post Body'].apply(tokenize_spacy)
df21['Sentences'] = df21['Cleaned Post Body'].apply(tokenize_sentences)
df21['Lemmas'] = df21['Cleaned Post Body'].apply(lemmatize)
# df['Stems'] = df['Cleaned Post Body'].apply(stem)

# Display the DataFrame to check results
print(df21)

       post_id                                                tag  \
0         7074  [types, types, types, types, string, string, s...   
1        11532                                         [php, php]   
2       148403  [c++, c++, unicode, unicode, stl, stl, utf-8, ...   
3       241269  [smtp, smtp, testing, testing, string, string,...   
4       271526  [nan, nan, java, java, nullpointerexception, n...   
...        ...                                                ...   
8901  78360036  [ip-camera, ip-camera, html5-video, html5-vide...   
8902  78360040  [python, python, celery, celery, django, djang...   
8903  78360041               [java, java, apache-poi, apache-poi]   
8904  78360045  [datetime, datetime, datetime, datetime, datet...   
8905  78360046  [dotnet-new, dotnet-new, dotnet-new, dotnet-ne...   

                                             post_title  \
0     What is the difference between String and stri...   
1      How can I find unused functions in a PHP proje

In [19]:
df28 = group_tags_with_other_columns(df28)
# Apply the functions to the DataFrame
df28['Cleaned Post Body'] = df28['post_body'].apply(clean_text)
df28['Code'] = df28['post_body'].apply(extract_code)
df28['Simple Tokens'] = df28['Cleaned Post Body'].apply(lambda text: text.split())
df28['NLTK Tokens'] = df28['Cleaned Post Body'].apply(word_tokenize)
# df['SpaCy Tokens'] = df['Cleaned Post Body'].apply(tokenize_spacy)
df28['Sentences'] = df28['Cleaned Post Body'].apply(tokenize_sentences)
df28['Lemmas'] = df28['Cleaned Post Body'].apply(lemmatize)
# df['Stems'] = df['Cleaned Post Body'].apply(stem)

# Display the DataFrame to check results
print(df28)

       post_id                                                tag  \
0        28796                            [[python, refactoring]]   
1        52550                   [[c, comma-operator, operators]]   
2        82875                   [[database-schema, sqlite, sql]]   
3       216202                     [[environment-variables, ssh]]   
4       231767             [[iterator, yield, generator, python]]   
...        ...                                                ...   
8696  78396679                 [[pytorch, cpu, dll, pip, python]]   
8697  78396683  [[java, intellij-idea, microstream, micronaut,...   
8698  78396684  [[flask, flask, flask, flask, flask, javascrip...   
8699  78396687          [[code-generation, kiota, java, openapi]]   
8700  78396693                                [[pyspark, python]]   

                                             post_title  \
0         What refactoring tools do you use for Python?   
1                    What does the comma operator , d

# Similarity Analysis with SBERT

## April 1-7

In [32]:
# Load your DataFrame
# Assuming df is already loaded and 'Cleaned Post Body' is prepared

# Load pre-trained model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
start_time = time()
embeddings = model.encode(df7['Cleaned_Post_Body'].tolist(), show_progress_bar=True, convert_to_tensor=False)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]  # Normalize for cosine similarity

# Function to find similar posts using dot product for cosine similarity
def find_similar_posts_cosine(index, top_k=3):
    # Compute cosine similarity with all posts
    cos_similarities = np.dot(embeddings, embeddings[index].T)
    # Get the indices of the posts with the highest similarity scores
    top_indices = np.argsort(-cos_similarities)[1:top_k+1]  # Exclude the query itself
    top_scores = cos_similarities[top_indices]
    return top_indices, top_scores

# Apply the function to each post and handle absence of similar posts
def get_similar_posts_info_sbert(row_idx):
    similar_indices, similarity_scores = find_similar_posts_cosine(row_idx, 3)
    if len(similar_indices) == 0:
        return "None"
    else:
        return ', '.join(f"id {df7.loc[sim_idx, 'post_id']}: {score:.4f}" for sim_idx, score in zip(similar_indices, similarity_scores))

# Adding a new column for similar posts info
df7['Similar_Post_IDs_and_Scores_SBert'] = df7.index.map(get_similar_posts_info_sbert)

# Execution time
execution_time = time() - start_time
print(f"Execution Time: {execution_time / 60:.2f} minutes")

# Print progress and results for review
for idx, row in df7.iterrows():
    print(f"Processing row {idx+1}/{len(df7)}")
    print(row['Similar_Post_IDs_and_Scores_SBert'])

# Final DataFrame output
print(df7[['post_id', 'Similar_Post_IDs_and_Scores_SBert']])


Batches:   0%|          | 0/253 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Processing row 5593/8085
id 78232012: 0.5127, id 78242690: 0.5123, id 78269731: 0.4952
Processing row 5594/8085
id 77325093: 0.5428, id 78255727: 0.5048, id 77843961: 0.4964
Processing row 5595/8085
id 78263696: 0.4134, id 78256318: 0.3854, id 78230792: 0.3794
Processing row 5596/8085
id 48306823: 0.4747, id 49732232: 0.4596, id 78173164: 0.4488
Processing row 5597/8085
id 78271463: 0.4603, id 78272001: 0.4524, id 78281832: 0.4317
Processing row 5598/8085
id 56568304: 0.5565, id 78269258: 0.5520, id 78266234: 0.5292
Processing row 5599/8085
id 78256014: 0.6302, id 78260089: 0.5865, id 78269926: 0.5088
Processing row 5600/8085
id 78277130: 0.5788, id 78256957: 0.5212, id 78261879: 0.4979
Processing row 5601/8085
id 76236300: 0.4431, id 78261588: 0.4409, id 8430019: 0.4395
Processing row 5602/8085
id 78256620: 0.6289, id 78283091: 0.5755, id 78272003: 0.5356
Processing row 5603/8085
id 78273181: 0.4745, id 78285456: 0.4231, id 78252933: 

In [33]:
df7.to_csv("Week_04-07-2024_similarity")

In [ ]:
df7

,post_id,tag,post_title,post_body,post_date,Cleaned_Post_Body,Code,Cleaned Post Body,Simple Tokens,NLTK Tokens,Sentences,Lemmas,Similar_Post_IDs_and_Scores_SBert
0,597687,"[refactoring, vim, variables]",How to quickly change variable names in Vim?,<p>I am using Vim to read through a lot of C a...,2009-02-28 07:29:54+00:00,I am using Vim to read through a lot of C and ...,,I am using Vim to read through a lot of C and ...,"[I, am, using, Vim, to, read, through, a, lot,...","[I, am, using, Vim, to, read, through, a, lot,...",[I am using Vim to read through a lot of C and...,"[I, be, use, Vim, to, read, through, a, lot, o...","id 61664546: 0.5397, id 78062513: 0.4721, id 7..."
1,598951,"[javascript, html]",What is the easiest way to detect if at least ...,<p>I have an HTML form with over 20 fields. I...,2009-03-01 00:15:30+00:00,I have an HTML form with over fields. I also h...,,I have an HTML form with over fields. I also h...,"[I, have, an, HTML, form, with, over, fields.,...","[I, have, an, HTML, form, with, over, fields, ...","[I have an HTML form with over fields, I also ...","[I, have, an, html, form, with, over, field, ....","id 10311663: 0.6076, id 30995409: 0.5342, id 6..."
2,761194,"[abstract-class, oop, interface]",Interface vs Abstract Class (general OO),<p>I have recently had two telephone interview...,2009-04-17 16:42:38+00:00,I have recently had two telephone interviews w...,,I have recently had two telephone interviews w...,"[I, have, recently, had, two, telephone, inter...","[I, have, recently, had, two, telephone, inter...",[I have recently had two telephone interviews ...,"[I, have, recently, have, two, telephone, inte...","id 78302962: 0.5038, id 78290995: 0.4687, id 7..."
3,763516,"[sqlite, schema, database]",information_schema.columns on sqlite,<p>I would like to do a query like next:</p>\n...,2009-04-18 14:31:15+00:00,I would like to do a query like next SELECT ta...,"SELECT table_name, column_name, data_type, is_...",I would like to do a query like next SELECT ta...,"[I, would, like, to, do, a, query, like, next,...","[I, would, like, to, do, a, query, like, next,...",[I would like to do a query like next SELECT t...,"[I, would, like, to, do, a, query, like, next,...","id 78302357: 0.5321, id 78238842: 0.4954, id 7..."
4,783818,"[exception, javascript]",How do I create a custom Error in JavaScript?,<p>For some reason it looks like constructor d...,2009-04-23 22:35:21+00:00,For some reason it looks like constructor dele...,function NotImplementedError() { \n Error.app...,For some reason it looks like constructor dele...,"[For, some, reason, it, looks, like, construct...","[For, some, reason, it, looks, like, construct...",[For some reason it looks like constructor del...,"[for, some, reason, it, look, like, constructo...","id 78320869: 0.5013, id 78308420: 0.4495, id 7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8406,78322305,"[spring-data-jpa, postgresql, java, spring-boo...",Can't run several test at once using Testconta...,<p>I want to test persisting layer in my Sprin...,2024-04-13 23:47:26+00:00,I want to test persisting layer in my Spring B...,org.postgresql.util.PSQLException: An I/O erro...,I want to test persisting layer in my Spring B...,"[I, want, to, test, persisting, layer, in, my,...","[I, want, to, test, persisting, layer, in, my,...",[I want to test persisting layer in my Spring ...,"[I, want, to, test, persist, layer, in, my, Sp...","id 78318084: 0.5378, id 78293652: 0.5371, id 7..."
8407,78322310,"[authlogic, ruby-on-rails, ruby]",Rails 7 and authlogic 6.4.3 error with tests,<p>i'm trying to write some controller tests w...,2024-04-13 23:50:12+00:00,im trying to write some controller tests which...,Authlogic::Session::Activation::NotActivatedEr...,im trying to write some controller tests which...,"[im, trying, to, write, some, controller, test...","[im, trying, to, write, some, controller, test...",[im trying to write some controller tests whic...,"[I, m, try, to, write, so

## April 8-14

In [ ]:
# Load pre-trained model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
start_time = time()
embeddings = model.encode(df14['Cleaned_Post_Body'].tolist(), show_progress_bar=True, convert_to_tensor=False)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]  # Normalize for cosine similarity

# Apply the function to each post and handle absence of similar posts
def get_similar_posts_info_sbert(row_idx):
    similar_indices, similarity_scores = find_similar_posts_cosine(row_idx, 3)
    if len(similar_indices) == 0:
        return "None"
    else:
        return ', '.join(f"id {df14.loc[sim_idx, 'post_id']}: {score:.4f}" for sim_idx, score in zip(similar_indices, similarity_scores))

# Adding a new column for similar posts info
df14['Similar_Post_IDs_and_Scores_SBert'] = df14.index.map(get_similar_posts_info_sbert)

# Execution time
execution_time = time() - start_time
print(f"Execution Time: {execution_time / 60:.2f} minutes")

# Print progress and results for review
for idx, row in df14.iterrows():
    print(f"Processing row {idx+1}/{len(df14)}")
    print(row['Similar_Post_IDs_and_Scores_SBert'])

# Final DataFrame output
print(df14[['post_id', 'Similar_Post_IDs_and_Scores_SBert']])


Batches:   0%|          | 0/263 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Processing row 5919/8411
id 78299368: 0.4568, id 78321711: 0.4380, id 78290937: 0.4122
Processing row 5920/8411
id 69367552: 0.4727, id 78312877: 0.4299, id 78263454: 0.4202
Processing row 5921/8411
id 78309858: 0.4855, id 78321789: 0.4853, id 78318362: 0.4801
Processing row 5922/8411
id 78300070: 0.6889, id 78299775: 0.6559, id 78303542: 0.6139
Processing row 5923/8411
id 78297471: 0.4378, id 78294147: 0.4148, id 76620808: 0.4122
Processing row 5924/8411
id 78312873: 0.3832, id 36786066: 0.3443, id 78299538: 0.3202
Processing row 5925/8411
id 78278434: 0.4720, id 78311520: 0.4442, id 78320450: 0.3973
Processing row 5926/8411
id 78283248: 0.4354, id 78197537: 0.3742, id 78318897: 0.3720
Processing row 5927/8411
id 78293430: 0.5066, id 78298287: 0.5025, id 78301585: 0.4790
Processing row 5928/8411
id 78301878: 0.5560, id 78317175: 0.4986, id 78301915: 0.4939
Processing row 5929/8411
id 78316887: 0.5056, id 78296938: 0.3903, id 14605765:

In [ ]:
df14.head()

,post_id,tag,post_title,post_body,post_date,Cleaned_Post_Body,Code,Cleaned Post Body,Simple Tokens,NLTK Tokens,Sentences,Lemmas,Similar_Post_IDs_and_Scores_SBert
0,597687,"[refactoring, vim, variables]",How to quickly change variable names in Vim?,<p>I am using Vim to read through a lot of C a...,2009-02-28 07:29:54+00:00,I am using Vim to read through a lot of C and ...,,I am using Vim to read through a lot of C and ...,"[I, am, using, Vim, to, read, through, a, lot,...","[I, am, using, Vim, to, read, through, a, lot,...",[I am using Vim to read through a lot of C and...,"[I, be, use, Vim, to, read, through, a, lot, o...","id 61664546: 0.5397, id 78062513: 0.4721, id 7..."
1,598951,"[javascript, html]",What is the easiest way to detect if at least ...,<p>I have an HTML form with over 20 fields. I...,2009-03-01 00:15:30+00:00,I have an HTML form with over fields. I also h...,,I have an HTML form with over fields. I also h...,"[I, have, an, HTML, form, with, over, fields.,...","[I, have, an, HTML, form, with, over, fields, ...","[I have an HTML form with over fields, I also ...","[I, have, an, html, form, with, over, field, ....","id 10311663: 0.6076, id 30995409: 0.5342, id 6..."
2,761194,"[abstract-class, oop, interface]",Interface vs Abstract Class (general OO),<p>I have recently had two telephone interview...,2009-04-17 16:42:38+00:00,I have recently had two telephone interviews w...,,I have recently had two telephone interviews w...,"[I, have, recently, had, two, telephone, inter...","[I, have, recently, had, two, telephone, inter...",[I have recently had two telephone interviews ...,"[I, have, recently, have, two, telephone, inte...","id 78302962: 0.5038, id 78290995: 0.4687, id 7..."
3,763516,"[sqlite, schema, database]",information_schema.columns on sqlite,<p>I would like to do a query like next:</p>\n...,2009-04-18 14:31:15+00:00,I would like to do a query like next SELECT ta...,"SELECT table_name, column_name, data_type, is_...",I would like to do a query like next SELECT ta...,"[I, would, like, to, do, a, query, like, next,...","[I, would, like, to, do, a, query, like, next,...",[I would like to do a query like next SELECT t...,"[I, would, like, to, do, a, query, like, next,...","id 78302357: 0.5321, id 78238842: 0.4954, id 7..."
4,783818,"[exception, javascript]",How do I create a custom Error in JavaScript?,<p>For some reason it looks like constructor d...,2009-04-23 22:35:21+00:00,For some reason it looks like constructor dele...,function NotImplementedError() { \n Error.app...,For some reason it looks like constructor dele...,"[For, some, reason, it, looks, like, construct...","[For, some, reason, it, looks, like, construct...",[For some reason it looks like constructor del...,"[for, some, reason, it, look, like, constructo...","id 78320869: 0.5013, id 78308420: 0.4495, id 7..."


## April 15-21

In [20]:
start_time = time()
embeddings = model.encode(df21['Cleaned_Post_Body'].tolist(), show_progress_bar=True, convert_to_tensor=False)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]  # Normalize for cosine similarity

# Function to find similar posts using dot product for cosine similarity
def find_similar_posts_cosine(index, top_k=3):
    # Compute cosine similarity with all posts
    cos_similarities = np.dot(embeddings, embeddings[index].T)
    # Get the indices of the posts with the highest similarity scores
    top_indices = np.argsort(-cos_similarities)[1:top_k+1]  # Exclude the query itself
    top_scores = cos_similarities[top_indices]
    return top_indices, top_scores

# Apply the function to each post and handle absence of similar posts
def get_similar_posts_info_sbert(row_idx):
    similar_indices, similarity_scores = find_similar_posts_cosine(row_idx, 3)
    if len(similar_indices) == 0:
        return "None"
    else:
        return ', '.join(f"id {df21.loc[sim_idx, 'post_id']}: {score:.4f}" for sim_idx, score in zip(similar_indices, similarity_scores))

# Adding a new column for similar posts info
df21['Similar_Post_IDs_and_Scores_SBert'] = df21.index.map(get_similar_posts_info_sbert)

# Execution time
execution_time = time() - start_time
print(f"Execution Time: {execution_time / 60:.2f} minutes")

# Print progress and results for review
for idx, row in df21.iterrows():
    print(f"Processing row {idx+1}/{len(df21)}")
    print(row['Similar_Post_IDs_and_Scores_SBert'])

# Final DataFrame output
print(df21[['post_id', 'Similar_Post_IDs_and_Scores_SBert']])

Batches:   0%|          | 0/279 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Processing row 6414/8906
id 78345383: 0.4341, id 78347277: 0.3812, id 78328798: 0.3464
Processing row 6415/8906
id 78355012: 0.4320, id 78347566: 0.3994, id 78359863: 0.3980
Processing row 6416/8906
id 78341777: 0.5325, id 78330589: 0.4952, id 78347087: 0.4617
Processing row 6417/8906
id 78347474: 0.5792, id 78355357: 0.4502, id 78350160: 0.4403
Processing row 6418/8906
id 78287491: 0.4502, id 78284057: 0.4434, id 78324895: 0.4068
Processing row 6419/8906
id 78337023: 0.5408, id 78324999: 0.4889, id 78335663: 0.4881
Processing row 6420/8906
id 78344341: 0.5129, id 78348488: 0.4998, id 78339192: 0.4452
Processing row 6421/8906
id 40274153: 0.4707, id 78311623: 0.4554, id 78356228: 0.4454
Processing row 6422/8906
id 78333951: 0.4943, id 78343717: 0.4882, id 78343451: 0.4645
Processing row 6423/8906
id 78355280: 0.6676, id 78330199: 0.5921, id 78337972: 0.5586
Processing row 6424/8906
id 78339087: 0.4975, id 14340679: 0.4507, id 78344349:

In [21]:
df21.head()

,post_id,tag,post_title,post_body,post_date,Cleaned_Post_Body,Code,Cleaned Post Body,Simple Tokens,NLTK Tokens,Sentences,Lemmas,Similar_Post_IDs_and_Scores_SBert
0,7074,"[types, types, types, types, string, string, s...",What is the difference between String and stri...,"<p>What are the differences between these two,...",2008-08-10 07:18:02+00:00,What are the differences between these two and...,string s = &quot;Hello world!&quot;;\nString s...,What are the differences between these two and...,"[What, are, the, differences, between, these, ...","[What, are, the, differences, between, these, ...",[What are the differences between these two an...,"[what, be, the, difference, between, these, tw...","id 78342153: 0.4950, id 78346523: 0.4477, id 7..."
1,11532,"[php, php]",How can I find unused functions in a PHP project,<p>How can I find any unused functions in a PH...,2008-08-14 19:08:21+00:00,How can I find any unused functions in a PHP p...,token_get_all(),How can I find any unused functions in a PHP p...,"[How, can, I, find, any, unused, functions, in...","[How, can, I, find, any, unused, functions, in...",[How can I find any unused functions in a PHP ...,"[how, can, I, find, any, unused, function, in,...","id 1307376: 0.5891, id 78328938: 0.4248, id 78..."
2,148403,"[c++, c++, unicode, unicode, stl, stl, utf-8, ...",UTF8 to/from wide char conversion in STL,<p>Is it possible to convert UTF8 string in a ...,2008-09-29 12:03:23+00:00,Is it possible to convert UTF string in a stds...,,Is it possible to convert UTF string in a stds...,"[Is, it, possible, to, convert, UTF, string, i...","[Is, it, possible, to, convert, UTF, string, i...",[Is it possible to convert UTF string in a std...,"[be, it, possible, to, convert, utf, string, i...","id 78340612: 0.5437, id 78346070: 0.4425, id 7..."
3,241269,"[smtp, smtp, testing, testing, string, string,...",Method for email testing,<p>I am writing a program that will be emailin...,2008-10-27 20:26:54+00:00,I am writing a program that will be emailing r...,,I am writing a program that will be emailing r...,"[I, am, writing, a, program, that, will, be, e...","[I, am, writing, a, program, that, will, be, e...",[I am writing a program that will be emailing ...,"[I, be, write, a, program, that, will, be, ema...","id 78088240: 0.4323, id 78347230: 0.4023, id 7..."
4,271526,"[nan, nan, java, java, nullpointerexception, n...",How do I avoid checking for nulls in Java?,<p>I use <code>x != null</code> to avoid <a hr...,2008-11-07 08:31:40+00:00,I use x null to avoid NullPointerException. Is...,x != null\nNullPointerException\nif (x != null...,I use x null to avoid NullPointerException. Is...,"[I, use, x, null, to, avoid, NullPointerExcept...","[I, use, x, null, to, avoid, NullPointerExcept...","[I use x null to avoid NullPointerException, I...","[I, use, x, null, to, avoid, nullpointerexcept...","id 78340345: 0.5186, id 78352917: 0.4773, id 7..."


In [22]:
df21.to_csv("Week_04-21-2024_similarity")

## April 22-28

In [23]:
start_time = time()
embeddings = model.encode(df28['Cleaned_Post_Body'].tolist(), show_progress_bar=True, convert_to_tensor=False)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]  # Normalize for cosine similarity

# Function to find similar posts using dot product for cosine similarity
def find_similar_posts_cosine(index, top_k=3):
    # Compute cosine similarity with all posts
    cos_similarities = np.dot(embeddings, embeddings[index].T)
    # Get the indices of the posts with the highest similarity scores
    top_indices = np.argsort(-cos_similarities)[1:top_k+1]  # Exclude the query itself
    top_scores = cos_similarities[top_indices]
    return top_indices, top_scores

# Apply the function to each post and handle absence of similar posts
def get_similar_posts_info_sbert(row_idx):
    similar_indices, similarity_scores = find_similar_posts_cosine(row_idx, 3)
    if len(similar_indices) == 0:
        return "None"
    else:
        return ', '.join(f"id {df28.loc[sim_idx, 'post_id']}: {score:.4f}" for sim_idx, score in zip(similar_indices, similarity_scores))

# Adding a new column for similar posts info
df28['Similar_Post_IDs_and_Scores_SBert'] = df28.index.map(get_similar_posts_info_sbert)

# Execution time
execution_time = time() - start_time
print(f"Execution Time: {execution_time / 60:.2f} minutes")

# Print progress and results for review
for idx, row in df28.iterrows():
    print(f"Processing row {idx+1}/{len(df21)}")
    print(row['Similar_Post_IDs_and_Scores_SBert'])

# Final DataFrame output
print(df28[['post_id', 'Similar_Post_IDs_and_Scores_SBert']])

Batches:   0%|          | 0/272 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Processing row 6209/8906
id 78391512: 0.4231, id 78364701: 0.4086, id 78382188: 0.4084
Processing row 6210/8906
id 78385092: 0.6299, id 78370164: 0.5195, id 30970068: 0.4478
Processing row 6211/8906
id 78364917: 0.5175, id 78374821: 0.5073, id 78380748: 0.4424
Processing row 6212/8906
id 78374982: 0.5446, id 78378215: 0.5136, id 70876129: 0.5104
Processing row 6213/8906
id 78381305: 0.7543, id 78387770: 0.7219, id 78365403: 0.7148
Processing row 6214/8906
id 78393129: 0.5846, id 78392324: 0.5067, id 78371222: 0.5057
Processing row 6215/8906
id 78375210: 0.4386, id 78188691: 0.4147, id 78371306: 0.3592
Processing row 6216/8906
id 78372431: 0.6399, id 78380761: 0.4917, id 78373471: 0.4614
Processing row 6217/8906
id 78388124: 0.6168, id 78389446: 0.5806, id 42462219: 0.5759
Processing row 6218/8906
id 78381686: 0.5670, id 78362632: 0.5552, id 78374146: 0.5356
Processing row 6219/8906
id 78386829: 0.4772, id 78387060: 0.4744, id 78367239:

In [24]:
df28.to_csv("Week_04-28-2024_similarity")

In [38]:
df28.head(5)

,post_id,tag,post_title,post_body,post_date,Cleaned_Post_Body,Code,Cleaned Post Body,Simple Tokens,NLTK Tokens,Sentences,Similar_Post_IDs_and_Scores_SBert
0,28796,"[[python, refactoring]]",What refactoring tools do you use for Python?,<p>I have a bunch of classes I want to rename....,2008-08-26 18:26:51+00:00,I have a bunch of classes I want to rename. So...,,I have a bunch of classes I want to rename. So...,"[I, have, a, bunch, of, classes, I, want, to, ...","[I, have, a, bunch, of, classes, I, want, to, ...","[I have a bunch of classes I want to rename, S...","id 78389652: 0.4445, id 78393150: 0.4354, id 7..."
1,52550,"[[c, comma-operator, operators]]","What does the comma operator , do?","<p>What does the <code>,</code> operator do in...",2008-09-09 18:34:09+00:00,What does the operator do in C,",",What does the operator do in C,"[What, does, the, operator, do, in, C]","[What, does, the, operator, do, in, C]",[What does the operator do in C],"id 78368786: 0.6095, id 18875207: 0.5063, id 7..."
2,82875,"[[database-schema, sqlite, sql]]",How can I list the tables in a SQLite database...,"<p>What SQL can be used to list the tables, an...",2008-09-17 12:59:53+00:00,What SQL can be used to list the tables and th...,ATTACH\nsqlite3,What SQL can be used to list the tables and th...,"[What, SQL, can, be, used, to, list, the, tabl...","[What, SQL, can, be, used, to, list, the, tabl...",[What SQL can be used to list the tables and t...,"id 78039596: 0.5884, id 78365762: 0.5861, id 7..."
3,216202,"[[environment-variables, ssh]]",Why does an SSH remote command get fewer envir...,<p>I have a command that runs fine if I ssh to...,2008-10-19 09:18:20+00:00,I have a command that runs fine if I ssh to a ...,"ssh user@IP &lt;command&gt;\n\nssh user@IP ""en...",I have a command that runs fine if I ssh to a ...,"[I, have, a, command, that, runs, fine, if, I,...","[I, have, a, command, that, runs, fine, if, I,...",[I have a command that runs fine if I ssh to a...,"id 78390222: 0.4955, id 78370442: 0.4128, id 7..."
4,231767,"[[iterator, yield, generator, python]]","What does the ""yield"" keyword do in Python?","<p>What functionality does the <a href=""https:...",2008-10-23 22:21:11+00:00,What functionality does the yield keyword in P...,"yield\ndef _get_child_candidates(self, distanc...",What functionality does the yield keyword in P...,"[What, functionality, does, the, yield, keywor...","[What, functionality, does, the, yield, keywor...",[What functionality does the yield keyword in ...,"id 78383633: 0.5326, id 78388707: 0.5249, id 7..."


# Similarity results

In [6]:
import pandas as pd
import re

def find_high_similarity_posts(df, similarity_threshold=0.8):
    # Initialize a list to store each row dictionary before creating DataFrame
    results_data = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Parse the Similar_Post_IDs_and_Scores_SBert field
        similar_posts = row['Similar_Post_IDs_and_Scores_SBert']
        # Find all matches in the string
        matches = re.findall(r'id (\d+): (\d+\.\d+)', similar_posts)

        # Filter matches where the similarity score is above the threshold
        for post_id, score in matches:
            score = float(score)
            if score > similarity_threshold:
                # Find the similar post's details
                similar_post_row = df[df['post_id'] == int(post_id)].iloc[0] if int(post_id) in df['post_id'].values else None
                if similar_post_row is not None:
                    # Create a dictionary for the current row and add to the list
                    results_data.append({
                        'post_id': row['post_id'],
                        'Cleaned_Post_Body': row['Cleaned_Post_Body'],
                        'Similar_Post_ID': post_id,
                        'Similarity_Score': score,
                        'Similar_Cleaned_Post_Body': similar_post_row['Cleaned_Post_Body']
                    })

    # Create a DataFrame from the list of dictionaries
    results_df = pd.DataFrame(results_data)
    # Return the DataFrame which can be directly displayed as a table in Jupyter Notebook
    return results_df

# Example usage:
# Assume df is your DataFrame loaded with the necessary data
# results_df = find_high_similarity_posts(df)
# print(results_df)


In [7]:
import pandas as pd

def remove_duplicate_pairs(df):
    # We'll use a set to keep track of seen pairs, enforcing a consistent order for the pair elements
    seen_pairs = set()
    unique_rows = []

    for index, row in df.iterrows():
        # Enforce a consistent order by always putting the smaller id first in the pair
        post_id = int(row['post_id'])
        similar_post_id = int(row['Similar_Post_ID'])
        ordered_pair = (min(post_id, similar_post_id), max(post_id, similar_post_id))

        if ordered_pair not in seen_pairs:
            seen_pairs.add(ordered_pair)
            unique_rows.append(row)

    # Create a new DataFrame from the list of unique rows
    return pd.DataFrame(unique_rows)


## April 1-7

In [ ]:
# Assuming results_df7 is the DataFrame returned by find_high_similarity_posts
results_df7 = find_high_similarity_posts(df7)
filtered_results_df7 = remove_duplicate_pairs(results_df7)

# Display the DataFrame in Jupyter to see it as a table
filtered_results_df7

,post_id,Cleaned_Post_Body,Similar_Post_ID,Similarity_Score,Similar_Cleaned_Post_Body
0,78192487,Im facing an issue with OCR where Im attemptin...,78281476,0.8003,I am working on an OCR project where I need to...
1,78253299,When I did pip install dlib I got following er...,78278272,0.8825,I want to use OpenCV for a python project and ...
2,78256257,I have installed docker in wsl windows and sta...,78282070,0.8065,Im new to Spring Boot Development in Visual St...
3,78261532,I have a date picker on my react app. Which so...,78265248,0.8325,I am using reactdatepicker. Here I have a cale...
4,78262748,We have a Legacy Web Application which was bui...,78277870,0.8136,I am getting exception while sending the mail ...
5,78263217,Im trying to create a game project with pygame...,78277325,0.8277,Im new to VS Code and I previously did a littl...
6,78264515,I would like to have the following code work b...,78265529,0.8110,My code doesnt work because the value of the v...
7,78264556,I populate data prop from App.js file and send...,78264889,0.9093,I populate data from App.js to CarDetails page...
9,78265137,I have created an react project here is the co...,78274653,0.9767,I have created an react project here is the co...
11,78265521,I have a collection of videos stored in an S b...,78265545,0.8560,Im seeking assistance with automating the conv...


In [8]:
# Assuming results_df7 is the DataFrame returned by find_high_similarity_posts
results_df7 = find_high_similarity_posts(df7)
filtered_results_df7 = remove_duplicate_pairs(results_df7)

# Display the DataFrame in Jupyter to see it as a table
filtered_results_df7

,post_id,Cleaned_Post_Body,Similar_Post_ID,Similarity_Score,Similar_Cleaned_Post_Body
0,78192487,Im facing an issue with OCR where Im attemptin...,78281476,0.8003,I am working on an OCR project where I need to...
1,78253299,When I did pip install dlib I got following er...,78278272,0.8825,I want to use OpenCV for a python project and ...
2,78256257,I have installed docker in wsl windows and sta...,78282070,0.8065,Im new to Spring Boot Development in Visual St...
3,78261532,I have a date picker on my react app. Which so...,78265248,0.8325,I am using reactdatepicker. Here I have a cale...
4,78262748,We have a Legacy Web Application which was bui...,78277870,0.8136,I am getting exception while sending the mail ...
5,78263217,Im trying to create a game project with pygame...,78277325,0.8277,Im new to VS Code and I previously did a littl...
6,78264515,I would like to have the following code work b...,78265529,0.8110,My code doesnt work because the value of the v...
7,78264556,I populate data prop from App.js file and send...,78264889,0.9093,I populate data from App.js to CarDetails page...
9,78265137,I have created an react project here is the co...,78274653,0.9767,I have created an react project here is the co...
11,78265521,I have a collection of videos stored in an S b...,78265545,0.8560,Im seeking assistance with automating the conv...


## April 8-14

In [ ]:
results_df14 = find_high_similarity_posts(df14)
filtered_results_df14 = remove_duplicate_pairs(results_df14)
# Display the DataFrame in Jupyter to see it as a table
filtered_results_df14

,post_id,Cleaned_Post_Body,Similar_Post_ID,Similarity_Score,Similar_Cleaned_Post_Body
0,18209676,can someone please tell me How to find antilog...,28922876,0.8114,I wanted to know if there is a function in jav...
2,61079545,Im trying to write my own authentication code....,61088988,0.9097,Im trying to write my own authentication middl...
4,75339253,I am using pyenv to select python .. pyenv glo...,78295602,0.8342,My pyenv python version is .. The system pytho...
5,78267481,I am uploading videos to YouTube programmatica...,78300839,0.9007,Followup on How to set video language when upl...
6,78276136,PyTorch unable to locate the CUDA version in m...,78299543,0.8419,This question is the continuation of my previo...
7,78276136,PyTorch unable to locate the CUDA version in m...,78289946,0.8003,Im trying to install PyTorch through Conda. Ho...
8,78289946,Im trying to install PyTorch through Conda. Ho...,78299543,0.8330,This question is the continuation of my previo...
10,78291425,Im working on an image editing application usi...,78291466,0.8761,Im working on an image editing feature using R...
12,78292900,Im encountering a challenging issue where my R...,78297121,0.8220,Im encountering a perplexing issue in my React...
13,78293386,I have the following query that now works corr...,78297471,0.9499,I have the following query select oc.cardnumbe...


## April 15-21

In [ ]:
results_df21 = find_high_similarity_posts(df21)
filtered_results_df21 = remove_duplicate_pairs(results_df21)
# Display the DataFrame to see it as a table
filtered_results_df21

,post_id,Cleaned_Post_Body,Similar_Post_ID,Similarity_Score,Similar_Cleaned_Post_Body
0,18209676,can someone please tell me How to find antilog...,28922876,0.8114,I wanted to know if there is a function in jav...
2,61079545,Im trying to write my own authentication code....,61088988,0.9097,Im trying to write my own authentication middl...
4,75339253,I am using pyenv to select python .. pyenv glo...,78295602,0.8342,My pyenv python version is .. The system pytho...
5,78267481,I am uploading videos to YouTube programmatica...,78300839,0.9007,Followup on How to set video language when upl...
6,78276136,PyTorch unable to locate the CUDA version in m...,78299543,0.8419,This question is the continuation of my previo...
7,78276136,PyTorch unable to locate the CUDA version in m...,78289946,0.8003,Im trying to install PyTorch through Conda. Ho...
8,78289946,Im trying to install PyTorch through Conda. Ho...,78299543,0.8330,This question is the continuation of my previo...
10,78291425,Im working on an image editing application usi...,78291466,0.8761,Im working on an image editing feature using R...
12,78292900,Im encountering a challenging issue where my R...,78297121,0.8220,Im encountering a perplexing issue in my React...
13,78293386,I have the following query that now works corr...,78297471,0.9499,I have the following query select oc.cardnumbe...


## April 22-28

In [40]:
# Assuming results_df7 is the DataFrame returned by find_high_similarity_posts
results_df28 = find_high_similarity_posts(df28)
filtered_results_df28 = remove_duplicate_pairs(results_df28)

# Display the DataFrame in Jupyter to see it as a table
filtered_results_df28

,post_id,Cleaned_Post_Body,Similar_Post_ID,Similarity_Score,Similar_Cleaned_Post_Body
0,45195363,I used the sphinxquickstart to set everything ...,55297443,0.8402,I am new to Sphinx and I want to write a docum...
1,45195363,I used the sphinxquickstart to set everything ...,63440589,0.8010,Im trying to use the basics of Sphinx to docum...
4,73897514,I already have a way to deploy resources in an...,77134880,0.8528,I have a repo with multiple resource configura...
6,78267317,Description of the bug I want to drop a ListBo...,78370450,0.9562,Description of the bug I am stuck for days... ...
7,78322320,Im developing a chatbot application using Reac...,78365540,0.9539,Im developing a chatbot using a Flask backend ...
8,78357161,ltdoctype htmlgt lthtml email datacssstrictgt ...,78370357,0.9943,ltdoctype htmlgt lthtml email datacssstrictgt ...
9,78358519,I have the following Spring Security configura...,78390536,0.8562,Bean public SecurityFilterChain securityFilter...
10,78364831,Im working with Boost.Locale in MSYSs ucrt env...,78377690,0.8506,How do I verify boostlocalegenerator in C I ha...
11,78365403,Im working for a project for university and I ...,78372457,0.8060,I was tring to scrape some information about t...
13,78365634,I am trying to load assemblies from a mono gam...,78369373,0.9682,I am trying to load assemblies from a mono gam...


### Usage


In [9]:
def display_related_posts(data, input_post_id):
    # Check if the post_id exists in the dataframe
    if input_post_id not in data['post_id'].values:
        print(f"No data found for post_id {input_post_id}")
        return None

    # Extract the original post's information
    input_row = data[data['post_id'] == input_post_id].iloc[0]
    print("Original Post:")
    print("Post ID:", input_post_id)
    print("Original Body:\n", input_row['post_body'])
    print("\nCleaned Body:\n", input_row['Cleaned Post Body'])
    print("\nSimilar Posts:")

    # Extract post IDs and their similarity scores
    similar_posts_data = input_row['Similar_Post_IDs_and_Scores_SBert']
    if similar_posts_data == "None":
        print("No similar posts found.")
        return

    # Create a DataFrame to store similar posts details
    results = pd.DataFrame(columns=['Post ID', 'Similarity Score', 'Original Body', 'Cleaned Body'])

    # Using regex to extract IDs and Scores
    similar_post_entries = re.findall(r'id (\d+): ([\d\.]+)', similar_posts_data)

    # Retrieve information for each similar post
    for post_id, sim_score in similar_post_entries:
        post_id = int(post_id)
        sim_score = float(sim_score)
        sim_post_row = data[data['post_id'] == post_id].iloc[0] if post_id in data['post_id'].values else None
        if sim_post_row is not None:
            new_index = len(results)
            results.loc[new_index] = [post_id, f"{sim_score:.4f}", sim_post_row['post_body'], sim_post_row['Cleaned Post Body']]
        else:
            new_index = len(results)
            results.loc[new_index] = [post_id, f"{sim_score:.4f}", 'Post not found', 'Post not found']

    return results

# Usage example:
post_id_to_query = 78271562  # Replace with your post_id
result_data = display_related_posts(df7, post_id_to_query)
if result_data is not None:
    # Using IPython display to show dataframe in table format
    from IPython.display import display
    display(result_data)

Original Post:
Post ID: 78271562
Original Body:
 <p>I'm making a simple 2d endless runner game for a project. The game is simple,</p>
<ul>
<li>it has one infinitely repeating obstacle,</li>
<li>it is supposed to have 3 difficulty levels that the player can choose.</li>
<li>The score starts at 0</li>
<li>the score updates when a collision is detected above the obstacle</li>
<li>and ends the game when the player hits the obstacle.</li>
</ul>
<p>(thanks to <a href="https://stackoverflow.com/users/16466946/kikon">kikon</a> for helping i modified your code a little)</p>
<p>I do have some other problems with my code I need to fix as well, but all help is appreciated so if any of you seeing this spot any other errors or questionable logic other than what I'm asking help for then please let me know.</p>
<p>Very sorry if it's just a small syntax error I missed. Here's the code:</p>
<p><div class="snippet" data-lang="js" data-hide="false" data-console="true" data-babel="false">
<div class="snipp

,Post ID,Similarity Score,Original Body,Cleaned Body
0,78281271,0.9875,<p>I'm making a simple 2d endless runner game ...,Im making a simple d endless runner game for a...
1,78279315,0.4856,<p>It must return the proper if else condition...,It must return the proper if else condition bu...
2,78283717,0.4730,<p>I'm writing a code for rock paper scissor.\...,Im writing a code for rock paper scissor. I ne...
